<h1 align='center'> 영상처리 프로그래밍 실습 7</h1>

<h6 align='right'> 2025. 5. 8. </h6>

<div class="alert alert-block alert-info">
    
- 파일 이름에서 00000000을 자신의 학번으로, name을 자신의 이름으로 수정하세요.

- 다음 줄에 자신의 이름, 학번, 학과(전공)을 적으세요.

* 이름: 이선재  &nbsp;&nbsp;          학번:  20227123  &nbsp;&nbsp;         학과(전공):
 빅데이터   
</div>

- JupyterLab 문서의 최신 버전은 [JupyterLab Documentation](https://jupyterlab.readthedocs.io/en/stable/index.html#/)을  참고하라

- Markdown은 [Markdown Guide](https://www.markdownguide.org/)를 참고하라.
- [Markdown Cheat Sheet](https://www.markdownguide.org/cheat-sheet/)

* 제출 마감: 5월 14일 (수) 오후 10:00까지 최종본을 SmartLEAD제출


In [1]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
print("OpenCV version", cv2.__version__)
print("NumPy version", np.__version__)

OpenCV version 4.10.0
NumPy version 1.26.4


## 문제 1.

지난 주에 작성했던 PyQt5를 이용한 GUI에 몇 가지 기능을 추가했던 프로그램을 실행한 후에 윈도우의 크기를 조정해 보자. 윈도우의 크기를 조정한 후에 영상의 크기도 윈도우의 크기에 비례해서 조정하려면 어떤 기능을 추가해야 하는지 조사해서 다시 작성하라.

In [16]:
import sys
from PyQt5.QtWidgets import (
    QApplication, 
    QMainWindow, 
    QLabel,
    QMenuBar, QAction,
    QFileDialog, QHBoxLayout, QWidget
)
from PyQt5.QtCore import Qt 
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈 
import numpy as np

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("단순 이미지 뷰어")
        self.setGeometry(100, 100, 1000, 600)

        # QLabel 두 개: 원본 / 변환
        self.original_label = QLabel("원본 이미지를 열어보세요")
        self.original_label.setAlignment(Qt.AlignCenter)

        self.processed_label = QLabel("변환된 이미지가 여기에 표시됩니다")
        self.processed_label.setAlignment(Qt.AlignCenter)

        # 수평 레이아웃
        self.central_widget = QWidget()
        self.hbox = QHBoxLayout(self.central_widget)
        self.hbox.addWidget(self.original_label)
        self.hbox.addWidget(self.processed_label)

        self.setCentralWidget(self.central_widget)

        self.current_image = None  # 원본 이미지 저장용 (NumPy)

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)

        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self, "이미지 열기", "", "Image Files (*.png *.jpg *.bmp *.jpeg *.tif *.tiff)"
        )
        if filename:
            image = image_tools.load_image(filename)  # RGB 형태로 반환
            self.current_image = image
            pixmap = self.numpy_to_qpixmap(image)
            self.original_label.setPixmap(pixmap.scaled(
                self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))
            self.processed_label.clear()

    def apply_log_transform(self):
        def log_transformation(image, f_max=255):
            C = 255 / np.log(1 + f_max)
            return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

        if self.current_image is None:
            return

        transformed = log_transformation(self.current_image, f_max=self.current_image.max())
        pixmap = self.numpy_to_qpixmap(transformed)
        self.processed_label.setPixmap(pixmap.scaled(
            self.processed_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
        ))

    def numpy_to_qpixmap(self, image):
        h, w, ch = image.shape
        bytes_per_line = ch * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

app = QApplication.instance()
if app is None:
    app = QApplication([])

viewer = BasicViewer()
viewer.show()

try:
    sys.exit(app.exec_())
except SystemExit:
    pass


#### 수정 후 프로그램
- 원 영상과 처리된 영상을 표시하는 두 개의 QLabel의 setSizePolicy 함수에 QSizePolicy.Ignored 인자를 전달
- resizeEvent 처리 함수를 추가하여 윈도우 크기가 변할 때마다 다시 영상의 크기를 조정하여 표시

In [4]:
import sys
from PyQt5.QtWidgets import (
    QApplication, 
    QMainWindow, 
    QLabel,
    QMenuBar, QAction,
    QFileDialog, QHBoxLayout, QWidget
)
from PyQt5.QtCore import Qt 
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈 
import numpy as np

class BasicViewer(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("단순 이미지 뷰어")
        self.setGeometry(100, 100, 1000, 600)

        # QLabel 두 개: 원본 / 변환
        self.original_label = QLabel("원본 이미지를 열어보세요")
        self.original_label.setAlignment(Qt.AlignCenter)

        self.processed_label = QLabel("변환된 이미지가 여기에 표시됩니다")
        self.processed_label.setAlignment(Qt.AlignCenter)

        # 수평 레이아웃
        self.central_widget = QWidget()
        self.hbox = QHBoxLayout(self.central_widget)
        self.hbox.addWidget(self.original_label)
        self.hbox.addWidget(self.processed_label)
        self.hbox.setStretch(0, 1)
        self.hbox.setStretch(1, 1)  # 두 QLabel이 균등하게 크기 조정되도록

        self.setCentralWidget(self.central_widget)

        self.current_image = None  # 원본 이미지
        self.processed_image = None  # 변환 이미지

        self.create_menu()

    def create_menu(self):
        menubar = self.menuBar()

        # 파일 메뉴
        file_menu = menubar.addMenu("파일")
        open_action = QAction("열기", self)
        open_action.triggered.connect(self.open_image_dialog)
        file_menu.addAction(open_action)

        exit_action = QAction("종료", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # 변환 메뉴
        transform_menu = menubar.addMenu("변환")
        log_action = QAction("로그 변환", self)
        log_action.triggered.connect(self.apply_log_transform)
        transform_menu.addAction(log_action)

    def open_image_dialog(self):
        filename, _ = QFileDialog.getOpenFileName(
            self, "이미지 열기", "", "Image Files (*.png *.jpg *.bmp *.jpeg *.tif *.tiff)"
        )
        if filename:
            image = image_tools.load_image(filename)  # RGB 형태로 반환
            self.current_image = image
            self.processed_image = None
            self.update_display_images()

    def apply_log_transform(self):
        def log_transformation(image, f_max=255):
            C = 255 / np.log(1 + f_max)
            return (C * np.log(1. + image)).round().clip(0, 255).astype(np.uint8)

        if self.current_image is None:
            return

        transformed = log_transformation(self.current_image, f_max=self.current_image.max())
        self.processed_image = transformed
        self.update_display_images()

    def numpy_to_qpixmap(self, image):
        if not isinstance(image, np.ndarray):
            raise ValueError("입력 이미지는 NumPy 배열이어야 합니다")
        
        h, w, ch = image.shape
        if ch != 3:
            raise ValueError("이미지는 3채널(RGB)이어야 합니다")
        
        # 연속적인 메모리 보장
        image = np.ascontiguousarray(image, dtype=np.uint8)
        
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def update_display_images(self):
        # 레이아웃 업데이트 강제
        self.central_widget.updateGeometry()
        self.central_widget.layout().update()

        if self.current_image is not None:
            pixmap = self.numpy_to_qpixmap(self.current_image)
            # QLabel의 실제 크기를 기준으로 스케일링
            scaled_pixmap = pixmap.scaled(
                self.original_label.size(),
                Qt.KeepAspectRatio,
                Qt.SmoothTransformation
            )
            self.original_label.setPixmap(scaled_pixmap)

        if self.processed_image is not None:
            pixmap = self.numpy_to_qpixmap(self.processed_image)
            scaled_pixmap = pixmap.scaled(
                self.processed_label.size(),
                Qt.KeepAspectRatio,
                Qt.SmoothTransformation
            )
            self.processed_label.setPixmap(scaled_pixmap)
        else:
            self.processed_label.clear()

    def resizeEvent(self, event):
        # 창 크기 변경 후 즉시 레이아웃과 이미지를 업데이트
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.hbox.activate()  # 레이아웃 강제 활성화
        self.update_display_images()

# 실행부
app = QApplication.instance()
if app is None:
    app = QApplication([])

viewer = BasicViewer()
viewer.show()

try:
    sys.exit(app.exec_())
except SystemExit:
    pass

### 문제 2. 

다음을 만족하는 함수 moving_average_filtering(filename)를 완성하라.

- filename으로 전달받은 파일이 존재하는지 조사해서 존재하지 않으면 오류 메시지를 콘솔에 출력하고 return 한다.
- 파일이 존재하면
  - 파일을 읽고, 원본 영상과 이동 평균 필터링된 영상을 나란히 화면에 표시한다.
  - 이동 평균 필터의 커널의 크기를 지정할 수 있는 trackBar interface를 제공한다. 단, trackBar로 지정하는 값의 범위는 1부터 51까지로 하되, 짝수일 경우 그보다 1 작은 홀수로 커널의 크기를 지정한다.
  - 지정된 커널의 크기를 이용해서 원 영상에 이동 평균 필터링이 적용된 영상을 구한다.

### 2.1 OpenCV GUI 기반

In [ ]:
import cv2
import os
import numpy as np

def moving_average_filtering(filename):
    """지정된 이미지 파일에 이동 평균 필터링을 적용하고 결과를 표시합니다."""
    # 파일 존재 여부 확인
    if not os.path.exists(filename):
        print(f"오류: 파일 {filename}이 존재하지 않습니다.")
        return

    # 이미지 로드 (BGR 형식)
    img = cv2.imread(filename, cv2.IMREAD_COLOR)
    if img is None:
        print(f"오류: 파일 {filename}을 로드할 수 없습니다.")
        return

    # 윈도우 생성
    window_name = "Moving Average Filtering"
    cv2.namedWindow(window_name)

    # 트랙바 콜백 함수 (아무 동작 안 함)
    def on_trackbar(val):
        pass

    # 트랙바 생성 (1~51)
    cv2.createTrackbar("Kernel Size", window_name, 1, 51, on_trackbar)

    while True:
        # 트랙바에서 커널 크기 읽기
        kernel_size = cv2.getTrackbarPos("Kernel Size", window_name)
        # 짝수면 1 작은 홀수로 조정, 최소 1 보장
        kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
        kernel_size = max(1, kernel_size)

        # 이동 평균 필터링 적용
        filtered_img = cv2.blur(img, (kernel_size, kernel_size))

        # 원본과 필터링된 이미지 나란히 표시
        combined_img = np.hstack((img, filtered_img))
        cv2.imshow(window_name, combined_img)

        # ESC 키로 종료
        key = cv2.waitKey(10)
        if key == 27:
            break

    cv2.destroyAllWindows()

# 테스트 실행
if __name__ == "__main__":
    test_filename = "bird.jpg"  # 테스트 이미지 파일 경로
    moving_average_filtering(test_filename)

### 2.2 PyQt 기반

In [15]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QSlider, QVBoxLayout
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈

class ImageFilterWindow(QMainWindow):
    def __init__(self, filename):
        super().__init__()
        self.setWindowTitle("Moving Average Filtering")
        self.setGeometry(100, 100, 1200, 600)

        # 이미지 로드
        self.original_image = image_tools.load_image(filename)  # RGB 형식
        self.filtered_image = self.original_image.copy()

        # 중앙 위젯과 레이아웃
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 이미지 표시용 레이아웃
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.filtered_label = QLabel("필터링된 이미지")
        self.filtered_label.setAlignment(Qt.AlignCenter)
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.filtered_label)
        self.main_layout.addLayout(self.image_layout)

        # 슬라이더 (커널 크기 조정)
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(51)
        self.slider.setValue(1)
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(5)
        self.slider.valueChanged.connect(self.update_filtered_image)
        self.main_layout.addWidget(self.slider)

        # 초기 이미지 표시
        self.update_display_images()

    def numpy_to_qpixmap(self, image):
        if not isinstance(image, np.ndarray):
            raise ValueError("입력 이미지는 NumPy 배열이어야 합니다")
        h, w, ch = image.shape
        if ch != 3:
            raise ValueError("이미지는 3채널(RGB)이어야 합니다")
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def update_filtered_image(self):
        # 슬라이더에서 커널 크기 읽기
        kernel_size = self.slider.value()
        # 짝수면 1 작은 홀수로 조정
        kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
        kernel_size = max(1, kernel_size)

        # 이동 평균 필터링 (BGR로 변환 후 다시 RGB로)
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)
        self.filtered_image = cv2.blur(img_bgr, (kernel_size, kernel_size))
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)

        # 이미지 업데이트
        self.update_display_images()

    def update_display_images(self):
        # 원본 이미지 표시
        pixmap = self.numpy_to_qpixmap(self.original_image)
        self.original_label.setPixmap(pixmap.scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
        ))

        # 필터링된 이미지 표시
        pixmap = self.numpy_to_qpixmap(self.filtered_image)
        self.filtered_label.setPixmap(pixmap.scaled(
            self.filtered_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
        ))

    def resizeEvent(self, event):
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def moving_average_filtering(filename):
    """지정된 이미지 파일에 이동 평균 필터링을 적용하고 결과를 표시합니다."""
    # 파일 존재 여부 확인
    if not os.path.exists(filename):
        print(f"오류: 파일 {filename}이 존재하지 않습니다.")
        return

    # PyQt 애플리케이션 실행
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    
    window = ImageFilterWindow(filename)
    window.show()
    
    try:
        sys.exit(app.exec_())
    except SystemExit:
        pass

# 테스트 실행
if __name__ == "__main__":
    test_filename = "bird.jpg"  # 테스트 이미지 파일 경로
    moving_average_filtering(test_filename)

## 문제 3.
강의에서 설명한 여러 가지 필터를 필터 메뉴에 추가하라.

In [ ]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QSlider, QVBoxLayout,
    QMenuBar, QMenu, QAction, QFileDialog
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈

class ImageFilterWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("이선재")
        self.setGeometry(100, 100, 1200, 600)

        # 초기 이미지 (빈 이미지로 시작)
        self.original_image = None
        self.filtered_image = None
        self.current_filter = "Mean"  # 기본 필터: Mean Filter

        # 메뉴 바 생성
        self.create_menu_bar()

        # 중앙 위젯과 레이아웃
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 이미지 표시용 레이아웃
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.filtered_label = QLabel("필터링된 이미지")
        self.filtered_label.setAlignment(Qt.AlignCenter)
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.filtered_label)
        self.main_layout.addLayout(self.image_layout)

        # 슬라이더 (커널 크기 조정)
        self.slider = QSlider(Qt.Horizontal)
        self.slider.setMinimum(1)
        self.slider.setMaximum(51)
        self.slider.setValue(3)  # 기본값 3
        self.slider.setTickPosition(QSlider.TicksBelow)
        self.slider.setTickInterval(5)
        self.slider.valueChanged.connect(self.update_filtered_image)
        self.main_layout.addWidget(self.slider)

    def create_menu_bar(self):
        """메뉴 바와 필터 메뉴를 생성하며 단축키 추가"""
        menubar = self.menuBar()
        
        # File 메뉴
        file_menu = menubar.addMenu("File")
        open_action = QAction("Open Image", self)
        open_action.setShortcut("Ctrl+O")  # 단축키: Ctrl+O
        open_action.triggered.connect(self.open_image)
        file_menu.addAction(open_action)
        
        # Filter 메뉴
        filter_menu = menubar.addMenu("Filter")
        filters = [
            ("Mean Filter", "Mean", "Ctrl+1"),
            ("Gaussian Blur", "Gaussian", "Ctrl+2"),
            ("Median Filter", "Median", "Ctrl+3"),
            ("Sobel Filter", "Sobel", "Ctrl+4"),
            ("Laplacian Filter", "Laplacian", "Ctrl+5")
        ]
        for filter_name, filter_id, shortcut in filters:
            action = QAction(filter_name, self)
            action.setShortcut(shortcut)  # 단축키 설정
            action.setData(filter_id)
            action.triggered.connect(self.set_filter)
            filter_menu.addAction(action)

    def open_image(self):
        """이미지 파일 열기"""
        filename, _ = QFileDialog.getOpenFileName(
            self, "Open Image", "", "Images (*.png *.jpg *.jpeg *.bmp)"
        )
        if filename:
            if not os.path.exists(filename):
                print(f"오류: 파일 {filename}이 존재하지 않습니다.")
                return
            try:
                self.original_image = image_tools.load_image(filename)  # RGB 형식
                self.filtered_image = self.original_image.copy()
                self.update_filtered_image()
                self.update_display_images()
            except Exception as e:
                print(f"오류: 파일 {filename}을 로드할 수 없습니다. {str(e)}")

    def set_filter(self):
        """선택된 필터로 current_filter 업데이트"""
        action = self.sender()
        self.current_filter = action.data()
        self.update_filtered_image()

    def numpy_to_qpixmap(self, image):
        """NumPy 배열을 QPixmap으로 변환"""
        if not isinstance(image, np.ndarray):
            raise ValueError("입력 이미지는 NumPy 배열이어야 합니다")
        h, w, ch = image.shape
        if ch != 3:
            raise ValueError("이미지는 3채널(RGB)이어야 합니다")
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def update_filtered_image(self):
        """선택된 필터를 적용하여 필터링된 이미지 업데이트"""
        if self.original_image is None:
            return

        # 슬라이더에서 커널 크기 읽기
        kernel_size = self.slider.value()
        # 짝수면 1 작은 홀수로 조정 (Sobel/Laplacian 제외)
        if self.current_filter not in ["Sobel", "Laplacian"]:
            kernel_size = kernel_size - 1 if kernel_size % 2 == 0 else kernel_size
            kernel_size = max(1, kernel_size)

        # BGR로 변환
        img_bgr = cv2.cvtColor(self.original_image, cv2.COLOR_RGB2BGR)

        # 필터 적용
        if self.current_filter == "Mean":
            self.filtered_image = cv2.blur(img_bgr, (kernel_size, kernel_size))
        elif self.current_filter == "Gaussian":
            self.filtered_image = cv2.GaussianBlur(img_bgr, (kernel_size, kernel_size), 0)
        elif self.current_filter == "Median":
            self.filtered_image = cv2.medianBlur(img_bgr, kernel_size)
        elif self.current_filter == "Sobel":
            # Sobel: x, y 방향 경계 검출 후 결합
            sobel_x = cv2.Sobel(img_bgr, cv2.CV_64F, 1, 0, ksize=3)
            sobel_y = cv2.Sobel(img_bgr, cv2.CV_64F, 0, 1, ksize=3)
            sobel_x = cv2.convertScaleAbs(sobel_x)
            sobel_y = cv2.convertScaleAbs(sobel_y)
            self.filtered_image = cv2.addWeighted(sobel_x, 0.5, sobel_y, 0.5, 0)
        elif self.current_filter == "Laplacian":
            # Laplacian: 2차 도함수로 경계 강조
            self.filtered_image = cv2.Laplacian(img_bgr, cv2.CV_64F, ksize=3)
            self.filtered_image = cv2.convertScaleAbs(self.filtered_image)

        # RGB로 변환
        self.filtered_image = cv2.cvtColor(self.filtered_image, cv2.COLOR_BGR2RGB)

        # 이미지 업데이트
        self.update_display_images()

    def update_display_images(self):
        """원본과 필터링된 이미지 표시"""
        if self.original_image is None:
            return

        # 원본 이미지 표시
        pixmap = self.numpy_to_qpixmap(self.original_image)
        self.original_label.setPixmap(pixmap.scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
        ))

        # 필터링된 이미지 표시
        if self.filtered_image is not None:
            pixmap = self.numpy_to_qpixmap(self.filtered_image)
            self.filtered_label.setPixmap(pixmap.scaled(
                self.filtered_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))

    def resizeEvent(self, event):
        """창 크기 조정 시 이미지 스케일링"""
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def moving_average_filtering(filename=None):
    """지정된 이미지 파일에 필터링을 적용하고 결과를 표시"""
    # PyQt 애플리케이션 실행
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    
    window = ImageFilterWindow()    
    if filename:
        if not os.path.exists(filename):
            print(f"오류: 파일 {filename}이 존재하지 않습니다.")
            return
        window.original_image = image_tools.load_image(filename)
        window.filtered_image = window.original_image.copy()
        window.update_filtered_image()
        window.update_display_images()
    window.show()
    
    try:
        sys.exit(app.exec_())
    except SystemExit:
        pass

# 테스트 실행
if __name__ == "__main__":
    moving_average_filtering()

### 문제 4.

다음 URL에서 'Lincoln.jpg',  'Lincoln_wave_1.png', 'Lincoln_wave_2.png' 파일을 로컬 드라이브에 다운로드하라.

https://drive.google.com/drive/u/0/folders/1zbjtkf9nHy9VniuLI4wHilbrN_JBvhYi

로컬 드라이브에서 'Lincoln.jpg' 파일을 읽고, 'Lincoln_wave_1.png' 또는 'Lincoln_wave_2.png'와 같은 그림을 화면에 표시하는 프로그램을 작성하라.

참고 사이트: https://mymodernmet.com/tyler-foust-one-line-drawing-portraits/

In [19]:
import sys
import os
import cv2
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QLabel, QHBoxLayout, QWidget, QVBoxLayout,
    QMenuBar, QMenu, QAction, QFileDialog
)
from PyQt5.QtCore import Qt
from PyQt5.QtGui import QPixmap, QImage
import image_tools  # 사용자 정의 모듈

class LincolnWaveWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Lincoln Wave Generator")
        self.setGeometry(100, 100, 1200, 600)

        # 초기 이미지
        self.original_image = None
        self.wave_image = None

        # 메뉴 바 생성
        self.create_menu_bar()

        # 중앙 위젯과 레이아웃
        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)
        self.main_layout = QVBoxLayout(self.central_widget)

        # 이미지 표시용 레이아웃
        self.image_layout = QHBoxLayout()
        self.original_label = QLabel("원본 이미지")
        self.original_label.setAlignment(Qt.AlignCenter)
        self.wave_label = QLabel("파형 이미지")
        self.wave_label.setAlignment(Qt.AlignCenter)
        self.image_layout.addWidget(self.original_label)
        self.image_layout.addWidget(self.wave_label)
        self.main_layout.addLayout(self.image_layout)

    def create_menu_bar(self):
        """메뉴 바 생성"""
        menubar = self.menuBar()
        
        # File 메뉴
        file_menu = menubar.addMenu("File")
        open_action = QAction("Open Image", self)
        open_action.setShortcut("Ctrl+O")
        open_action.triggered.connect(self.open_image)
        file_menu.addAction(open_action)
        
        save_action = QAction("Save Wave Image", self)
        save_action.setShortcut("Ctrl+S")
        save_action.triggered.connect(self.save_image)
        file_menu.addAction(save_action)

        # Wave 메뉴
        wave_menu = menubar.addMenu("Wave")
        horizontal_action = QAction("Horizontal Wave", self)
        horizontal_action.setShortcut("Ctrl+1")
        horizontal_action.triggered.connect(lambda: self.apply_wave("horizontal"))
        wave_menu.addAction(horizontal_action)

        vertical_action = QAction("Vertical Wave", self)
        vertical_action.setShortcut("Ctrl+2")
        vertical_action.triggered.connect(lambda: self.apply_wave("vertical"))
        wave_menu.addAction(vertical_action)

        both_action = QAction("Horizontal + Vertical Wave", self)
        both_action.setShortcut("Ctrl+3")
        both_action.triggered.connect(lambda: self.apply_wave("both"))
        wave_menu.addAction(both_action)

    def open_image(self):
        """이미지 파일 열기"""
        filename, _ = QFileDialog.getOpenFileName(
            self, "Open Image", "", "Images (*.png *.jpg *.jpeg *.bmp)"
        )
        if filename and os.path.exists(filename):
            try:
                self.original_image = image_tools.load_image(filename)
                self.wave_image = self.original_image.copy()
                self.update_display_images()
            except Exception as e:
                print(f"오류: 파일 {filename}을 로드할 수 없습니다. {str(e)}")

    def save_image(self):
        """파형 이미지를 저장"""
        if self.wave_image is not None:
            filename, _ = QFileDialog.getSaveFileName(
                self, "Save Image", "Lincoln_wave.png", "PNG (*.png)"
            )
            if filename:
                cv2.imwrite(filename, cv2.cvtColor(self.wave_image, cv2.COLOR_RGB2BGR))
                print(f"이미지가 {filename}으로 저장되었습니다.")

    def numpy_to_qpixmap(self, image):
        """NumPy 배열을 QPixmap으로 변환"""
        if not isinstance(image, np.ndarray):
            raise ValueError("입력 이미지는 NumPy 배열이어야 합니다")
        h, w, ch = image.shape
        if ch != 3:
            raise ValueError("이미지는 3채널(RGB)이어야 합니다")
        image = np.ascontiguousarray(image, dtype=np.uint8)
        bytes_per_line = 3 * w
        qimage = QImage(image.data, w, h, bytes_per_line, QImage.Format_RGB888)
        return QPixmap.fromImage(qimage)

    def apply_wave(self, wave_type):
        """파형 효과 적용"""
        if self.original_image is None:
            return

        h, w, _ = self.original_image.shape
        self.wave_image = self.original_image.copy()

        if wave_type in ["horizontal", "both"]:
            # 수평 파형
            for y in range(h):
                offset = int(20 * np.sin(2 * np.pi * y / 50))  # 사인파 오프셋
                for x in range(w):
                    if 0 <= x + offset < w:
                        self.wave_image[y, x] = self.original_image[y, x + offset]

        if wave_type in ["vertical", "both"]:
            # 수직 파형
            for x in range(w):
                offset = int(20 * np.sin(2 * np.pi * x / 50))  # 사인파 오프셋
                for y in range(h):
                    if 0 <= y + offset < h:
                        self.wave_image[y, x] = self.original_image[y + offset, x]

        self.update_display_images()

    def update_display_images(self):
        """원본과 파형 이미지 표시"""
        if self.original_image is None:
            return

        # 원본 이미지
        pixmap = self.numpy_to_qpixmap(self.original_image)
        self.original_label.setPixmap(pixmap.scaled(
            self.original_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
        ))

        # 파형 이미지
        if self.wave_image is not None:
            pixmap = self.numpy_to_qpixmap(self.wave_image)
            self.wave_label.setPixmap(pixmap.scaled(
                self.wave_label.size(), Qt.KeepAspectRatio, Qt.SmoothTransformation
            ))

    def resizeEvent(self, event):
        """창 크기 조정 시 이미지 스케일링"""
        super().resizeEvent(event)
        self.central_widget.updateGeometry()
        self.main_layout.activate()
        self.update_display_images()

def main():
    app = QApplication.instance()
    if app is None:
        app = QApplication(sys.argv)
    
    window = LincolnWaveWindow()
    window.show()
    
    try:
        sys.exit(app.exec_())
    except SystemExit:
        pass

if __name__ == "__main__":
    main()